In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import math
import time
from keras import backend as K
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.tree import ExtraTreeRegressor
from sklearn.metrics import r2_score
#from multiprocessing import Pool,cpu_count
import sklearn.metrics as m
from collections import Counter
from sklearn.ensemble import ExtraTreesRegressor,RandomForestRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

Using TensorFlow backend.


In [2]:
'''
Function that takes in a dataframe and then returns the dictionary of accuracies.
'''
def plot_test(y_test,y_pred,start,end,string):
    #Plot of testing.
    plt.figure(figsize=(10,10))
    plt.plot(y_test[start:end],'r')
    plt.plot(y_pred[start+1:end],'b')
    plt.xticks(np.arange(0,end-start,1))
    plt.legend()
    plt.title(string)
    plt.show()

def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_pred - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

def nn_model(localdf,localdict,times,metric,num_cols,hour_day):
    for counter in range(times):
            #i = num_cols
        #for i in range(num_cols):
            i=num_cols
            df = pd.DataFrame()
            if(i != 0):
                print(localdf.columns[i])
                
                ndf = localdf
                x = ndf[ndf.columns[0:ndf.shape[1]]]
                print ("\n",x.columns)
                y = ndf[ndf.columns[-1:]]
                
                print (x.shape)
                print (y.shape)
                
                #X_train,X_test,y_train,y_test = train_test_split(x.values,y.values,test_size=0.2,random_state=1)
                train_size = int(70*x.shape[0]/100)
                X_train = x.values[:train_size]
                y_train = y.values[:train_size]
                    
                X_test = x.values[train_size:]
                y_test = y.values[train_size:]
                model = Sequential()
                model.add(Dense(10,input_dim=ndf.shape[1],kernel_initializer='normal', activation='relu'))
                #Output Layer.
                model.add(Dense(1,kernel_initializer='normal'))
                #Compile model.
                model.compile(loss='mean_squared_error',optimizer='adam',metrics=[metric])
                #Fit the model.
                history = model.fit(X_train,y_train,epochs=200,batch_size=200,verbose=1)
                #Final Evaluation of the model.
                #model.evaluate returns the 'loss' and the 'metrics' for the model.
                scores = model.evaluate(X_test,y_test,verbose=1)
                y_pred = model.predict(X_test)
                y_pred[y_pred<0]=0
                print ("R2 Score : ",r2_score(y_test,y_pred))
                print ("MSE : ",mean_squared_error(y_test[:-1],y_pred[1:]))
                print ("MAE : ",mean_absolute_error(y_test[:-1],y_pred[1:]))
                #print (str(coeff_determination(y_test,y_pred)))
                print ("Metric : %.5f" % (scores[1]))
                localdict.setdefault(counter,[]).append((scores[1]))
                model.summary()
                weights1 = model.layers[0].get_weights()[0]
                biases1 = model.layers[0].get_weights()[1]
                weights2 = model.layers[1].get_weights()[0]
                biases2 = model.layers[1].get_weights()[1]
                plot_test(y_test,y_pred,0,50,"ANN")
    return localdict,history,y_test,y_pred,scores,weights1, weights2, biases1, biases2 

In [3]:
data = pd.read_pickle('../Datasets/Pecan Street/New_Data/load_data_1632.pkl')
for i in data.columns:
    if data[i].isnull().sum() >= data.values.shape[0]-10000 : del data[i]

In [4]:
data = pd.DataFrame(data.values[175:],columns=data.columns)
data = data*1000
data.head()

,dataid,localhour,use,air1,air2,air3,bathroom1,clotheswasher1,dishwasher1,disposal1,...,grid,kitchenapp1,kitchenapp2,lights_plugs1,lights_plugs2,microwave1,outsidelights_plugs1,outsidelights_plugs2,refrigerator1,waterheater1
0,1632000,2013-03-23 08:00:00-052013-03-23 08:00:00-0520...,466.633,0,0,0,0,0,4.2,0,...,466.633,17.35,1.5,11,0,20.7167,0,95.8833,NaN,0
1,1632000,2013-03-23 09:00:00-052013-03-23 09:00:00-0520...,503.983,0,0,0,0,0,4.11667,0,...,503.983,5.31667,1.35,10.85,0,8,0,95.4167,NaN,0
2,1632000,2013-03-23 10:00:00-052013-03-23 10:00:00-0520...,561.033,0,0,0,0,0,4.06667,0,...,561.033,0.316667,1.45,9.26667,0,8,0,94.7167,NaN,181.567
3,1632000,2013-03-23 11:00:00-052013-03-23 11:00:00-0520...,1683.28,0,0,0,0,0,3.7,0,...,1683.28,0,1.95,7.6,0,8,0,93.05,NaN,1324.77
4,1632000,2013-03-23 12:00:00-052013-03-23 12:00:00-0520...,366.05,0,0,0,0,0,4.33333,0,...,366.05,0,0.9,0,0,8,0,95,NaN,0


In [5]:
data.tail()

,dataid,localhour,use,air1,air2,air3,bathroom1,clotheswasher1,dishwasher1,disposal1,...,grid,kitchenapp1,kitchenapp2,lights_plugs1,lights_plugs2,microwave1,outsidelights_plugs1,outsidelights_plugs2,refrigerator1,waterheater1
40815,1632000,2017-11-17 22:00:00-062017-11-17 22:00:00-0620...,1198.5,0,0,0,0,0,0.15,0,...,1198.5,0,0,0,0,144.283,6.36667,83.5167,85.7167,0
40816,1632000,2017-11-17 23:00:00-062017-11-17 23:00:00-0620...,619.4,0,0,0,0,0,0,0,...,619.4,0,0,0,0,56.4667,3.3,84.6167,110.633,0
40817,1632000,2017-11-18 00:00:00-062017-11-18 00:00:00-0620...,442.217,0,0,0,0,0,0,0,...,442.217,0,0,0,0,82.3167,3.45,84.6833,86.3667,0
40818,1632000,2017-11-18 01:00:00-062017-11-18 01:00:00-0620...,386.933,0,0,0,0,0,0,0,...,386.933,0,0,0,0,81.8,3.78333,85.0333,87.7833,0
40819,1632000,2017-11-18 02:00:00-062017-11-18 02:00:00-0620...,382.117,0,0,0,0,0,0,0,...,382.117,0,0,0,0,81.95,3.7,85.1667,82.6,0


In [6]:
data['air1'][34492] = (data['air1'][34492-168] + data['air1'][34492+168])/2
data['air2'][34492] = (data['air2'][34492-168] + data['air2'][34492+168])/2
data['grid'][34492] = (data['grid'][34492-168] + data['grid'][34492+168])/2
data['outsidelights_plugs1'][34492] = (data['outsidelights_plugs1'][34492-168] + data['outsidelights_plugs1'][34492+168])/2
data['outsidelights_plugs2'][34492] = (data['outsidelights_plugs2'][34492-168] + data['outsidelights_plugs2'][34492+168])/2
data['refrigerator1'][34492] = 0.0
x = data.values[34492][2:]
data['use'][34492] = sum(x[1:])

In [7]:
data.isnull().sum()

dataid                     0
localhour                  0
use                        0
air1                       0
air2                       0
air3                       0
bathroom1                  0
clotheswasher1             0
dishwasher1                0
disposal1                  0
dryg1                      0
furnace1                   0
furnace2                   0
garage1                    0
garage2                    0
grid                       0
kitchenapp1                0
kitchenapp2                0
lights_plugs1              0
lights_plugs2              0
microwave1                 0
outsidelights_plugs1       0
outsidelights_plugs2       0
refrigerator1           7767
waterheater1               0
dtype: int64

In [8]:
df_thermal = data[['air1','air2','air3','furnace1','furnace2','waterheater1']].copy()
df_thermal['tot'] = df_thermal['air1'] + df_thermal['air2'] + df_thermal['air3'] + df_thermal['furnace1'] + df_thermal['furnace2'] + df_thermal['waterheater1']
df_tot_ther = df_thermal[['tot']]

In [9]:
nhrs = int(df_tot_ther.values.shape[0]/24)
y = np.ndarray(shape=(nhrs,))
val = []
for i in range(nhrs):
    val = df_tot_ther.values[i:i+24]
    y[i] = sum(val)
y = y[7:]
x = df_tot_ther.values

In [ ]:
#leave empty to add proper nn model call fn

In [ ]:
'''weather = pd.read_csv("E:\\TherML\\Datasets\\Pecan Street\\Weather data\\weather_data.csv",dtype='object')
d = open("E:\\TherML\\Datasets\\Pecan Street\\Weather data\\labels_weather.txt",'r').read()
d = d.split('|')[3:]
d = [k.strip() for k in d]
t = [d[k] for k in range(len(d)) if k%2==1]
t.append(d[0])
t = t[-1:] + t[:-1]
t = [x for x in t if x != '']
weather.columns = t
weather.isnull().sum()
weather.to_csv("E:\\TherML\\Datasets\\Pecan Street\\Weather data\\weather_data1.csv", encoding='utf8', index=False)'''

In [10]:
weather = pd.read_csv("..\\Datasets\\Pecan Street\\Weather data\\weather_data1.csv",dtype='object')
for i in weather.columns:
    if weather[i].isnull().sum() >= weather.values.shape[0]-60000 : del weather[i]
'''
t1 = weather['latitude'].str.contains('30.')
t2 = [c for c in range(len(t1)) if t1[c] == True]
t2[-1]
'''
weather = pd.DataFrame(weather.values[19495:58821], columns=weather.columns)
weather.isnull().sum()

timestamp                  0
latitude                   0
longitude                  0
tz_offset                  0
Label                      0
temperature                0
dew_point                  0
humidity                   0
visibility                 0
apparent_temperature       0
pressure                   0
wind_speed                46
cloud_cover             2234
wind_bearing             201
precip_intensity           0
precip_probability         0
dtype: int64

In [11]:
weather = pd.DataFrame(weather.iloc[:,1:], columns=weather.columns[1:])
for i in weather.columns:
    if(i!='timestamp'):
        print(i)
        weather[i] = pd.to_numeric(weather[i])
        if(weather[i].isnull().sum()!=0):
            for j in range(weather.values.shape[0]):
                if(np.isnan(weather[i][j])):
                    weather[i][j] = 0.0
weather.isnull().sum()

latitude
longitude
tz_offset
Label
temperature
dew_point
humidity
visibility
apparent_temperature
pressure
wind_speed


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


cloud_cover
wind_bearing
precip_intensity
precip_probability


latitude                0
longitude               0
tz_offset               0
Label                   0
temperature             0
dew_point               0
humidity                0
visibility              0
apparent_temperature    0
pressure                0
wind_speed              0
cloud_cover             0
wind_bearing            0
precip_intensity        0
precip_probability      0
dtype: int64

In [12]:
df_tot_ther = pd.DataFrame(df_tot_ther.values[:39326], columns=df_tot_ther.columns)
weather['total_load'] = df_tot_ther[['tot']]

In [13]:
weather.columns

Index(['latitude', 'longitude', 'tz_offset', 'Label', 'temperature',
       'dew_point', 'humidity', 'visibility', 'apparent_temperature',
       'pressure', 'wind_speed', 'cloud_cover', 'wind_bearing',
       'precip_intensity', 'precip_probability', 'total_load'],
      dtype='object')

In [14]:
thermal_dict1 = {}
model_history1 = {}

In [ ]:
ti = time.time()
thermal_dict1,model_history1,y_test1,y_pred1,scores,w1,w2,b1,b2 = nn_model(weather,thermal_dict1,1,"mean_squared_error",weather.shape[1]-1,0)
tf = time.time()
print("weights",w1,"biases",b1,"weights",w2,"biases",b2)
print ("Time Taken : ",tf-ti)
print (thermal_dict1)

total_load

 Index(['latitude', 'longitude', 'tz_offset', 'Label', 'temperature',
       'dew_point', 'humidity', 'visibility', 'apparent_temperature',
       'pressure', 'wind_speed', 'cloud_cover', 'wind_bearing',
       'precip_intensity', 'precip_probability', 'total_load'],
      dtype='object')
(39326, 16)
(39326, 1)
Epoch 1/200
27528/27528 [==============================] - 9s 344us/step - loss: 889980.4608 - mean_squared_error: 889980.4608
Epoch 2/200
27528/27528 [==============================] - 1s 32us/step - loss: 340701.8867 - mean_squared_error: 340701.8867
Epoch 3/200
27528/27528 [==============================] - 1s 31us/step - loss: 106540.7137 - mean_squared_error: 106540.7137
Epoch 4/200
27528/27528 [==============================] - 1s 33us/step - loss: 11961.4218 - mean_squared_error: 11961.4218
Epoch 5/200
27528/27528 [==============================] - 1s 37us/step - loss: 551.3575 - mean_squared_error: 551.3575
Epoch 6/200
27528/27528 [===========================

27528/27528 [==============================] - 1s 36us/step - loss: 0.0101 - mean_squared_error: 0.0101
Epoch 68/200
27528/27528 [==============================] - 1s 36us/step - loss: 0.0088 - mean_squared_error: 0.0088
Epoch 69/200
27528/27528 [==============================] - 1s 33us/step - loss: 0.0079 - mean_squared_error: 0.0079
Epoch 70/200
27528/27528 [==============================] - 1s 35us/step - loss: 0.0071 - mean_squared_error: 0.0071
Epoch 71/200
27528/27528 [==============================] - 1s 34us/step - loss: 0.0063 - mean_squared_error: 0.0063
Epoch 72/200
27528/27528 [==============================] - 1s 35us/step - loss: 0.0062 - mean_squared_error: 0.0062
Epoch 73/200
27528/27528 [==============================] - 1s 34us/step - loss: 0.0055 - mean_squared_error: 0.0055
Epoch 74/200
27528/27528 [==============================] - 1s 34us/step - loss: 0.0050 - mean_squared_error: 0.0050
Epoch 75/200
27528/27528 [==============================] - 1s 30us/step - lo

27528/27528 [==============================] - 1s 35us/step - loss: 0.0018 - mean_squared_error: 0.0018
Epoch 137/200
27528/27528 [==============================] - 1s 32us/step - loss: 0.0029 - mean_squared_error: 0.0029
Epoch 138/200
27528/27528 [==============================] - 1s 34us/step - loss: 0.0030 - mean_squared_error: 0.0030
Epoch 139/200
27528/27528 [==============================] - 1s 33us/step - loss: 0.0103 - mean_squared_error: 0.0103
Epoch 140/200
27528/27528 [==============================] - 1s 29us/step - loss: 0.0070 - mean_squared_error: 0.0070
Epoch 141/200
27528/27528 [==============================] - 1s 33us/step - loss: 0.1094 - mean_squared_error: 0.1094
Epoch 142/200
27528/27528 [==============================] - 1s 32us/step - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 143/200
27528/27528 [==============================] - 1s 34us/step - loss: 0.0377 - mean_squared_error: 0.0377
Epoch 144/200
27528/27528 [==============================] - 1s 36us/s